## Ingesting PDF

In [2]:
%pip install --q unstructured langchain langchain-community
%pip install --q "unstructured[all-docs]" ipywidgets tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

C:\Users\jahan\AppData\Local\Temp\ipykernel_1360\3632673901.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [4]:
local_path = "testfile.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [5]:
# Preview first page
Markdown(data[0].page_content)

'Tables, Charts, and Graphs with Examples from History, Economics, Education, Psychology, Urban Affairs and Everyday Life\n\nREVISED: MICHAEL LOLKUS 2018\n\nTables, Charts, and Graphs Basics\n\n\uf075 We use charts and graphs to visualize data.\n\n\uf075 This data can either be generated data, data gathered from\n\nan experiment, or data collected from some source.\n\n\uf075 A picture tells a thousand words so it is not a surprise that\n\nmany people use charts and graphs when explaining data.\n\nTypes of Visual Representations of Data\n\nTable of Yearly U.S. GDP by Industry (in millions of dollars)\n\nSource: U.S. Bureau of Labor Statistics\n\nYear\n\n2010\n\n2011\n\n2012\n\n2013\n\nAll Industries\n\n26093515\n\n27535971\n\n28663246\n\n29601191\n\nManufacturing\n\n4992521\n\n5581942\n\n5841608\n\n5953299\n\nFinance, Insurance, Real Estate, Rental, Leasing\n\n4522451\n\n4618678\n\n4797313\n\n5031881\n\nArts, Entertainment, Recreation, Accommodation, and Food Service\n\n964032\n\n101523

## Vector Embeddings

In [7]:
!ollama list

NAME                       ID              SIZE      MODIFIED     
nomic-embed-text:latest    0a109f422b47    274 MB    15 hours ago    
llama2:latest              78e26419b446    3.8 GB    15 hours ago    


In [8]:
# # Pull nomic-embed-text model from Ollama if you don't have it
# !ollama pull nomic-embed-text
# # List models again to confirm it's available
# !ollama list

In [9]:
# 1. First clean up any existing ChromaDB installations
%pip uninstall -y chromadb
%pip uninstall -y protobuf

# 2. Install specific versions known to work together
%pip install -q protobuf==3.20.3
%pip install -q chromadb==0.4.22  # Using a stable older version
%pip install -q langchain-ollama

# 3. Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

Found existing installation: chromadb 0.4.22
Uninstalling chromadb-0.4.22:
  Successfully uninstalled chromadb-0.4.22
Note: you may need to restart the kernel to use updated packages.
Found existing installation: protobuf 5.29.1
Uninstalling protobuf-5.29.1:
  Successfully uninstalled protobuf-5.29.1
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.68.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.29.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [11]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [12]:
pip install chromadb==0.4.22

  Using cached chromadb-0.4.22-py3-none-any.whl.metadata (7.3 kB)
  Using cached protobuf-5.29.1-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached chromadb-0.4.22-py3-none-any.whl (509 kB)
Using cached protobuf-5.29.1-cp310-abi3-win_amd64.whl (434 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
Note: you may need to restart the kernel to use updated packages.


In [13]:
# 1. First clean up any existing ChromaDB installations
#%pip uninstall -y chromadb
%pip uninstall -y protobuf

# 2. Install specific versions known to work together
%pip install -q protobuf==3.20.3
#%pip install -q chromadb==0.4.22  # Using a stable older version
%pip install -q langchain-ollama

# 3. Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# 4. Now reimport with the new versions
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# 5. Try creating the vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)

Found existing installation: protobuf 5.29.1
Uninstalling protobuf-5.29.1:
  Successfully uninstalled protobuf-5.29.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.68.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.29.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


## Retrieval

In [15]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [16]:
!ollama list

NAME                       ID              SIZE      MODIFIED     
nomic-embed-text:latest    0a109f422b47    274 MB    15 hours ago    
llama2:latest              78e26419b446    3.8 GB    15 hours ago    


In [17]:
# LLM from Ollama
local_model = "llama2"
llm = ChatOllama(model=local_model)

In [18]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [19]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [20]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
chain.invoke("From page 6 get the tabular data")

ResponseError: model requires more system memory (8.4 GiB) than is available (2.9 GiB)

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()